# 1- Definition

### 1.1- Project Overviw

### 1.2- Problem statement

### 1.3- Metrics

# 2- Analysis

### 2.1- Data Exploration

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os
import gc

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from functools import reduce



from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold


# Modeling
import lightgbm as lgb

# Splitting data
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

In [2]:
path = '/home/ralph/Documents/UDACITY/Machine_Learning/Capstone Project/Data/all/'
out_path = '/home/ralph/Documents/UDACITY/Machine_Learning/Capstone Project/output/'

#### 2.1.1- Import the data

## Tentando achar os melhores parametros

In [3]:
# Create a default model
model = lgb.LGBMModel()
model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [15]:
# Hyperparameter grid
param_grid = {
    'n_estimators': [500,5000 ],
    'class_weight' : ['balanced'],
    'boosting_type': ['gbdt', 'goss', 'dart'],
    'num_leaves': [20, 150],
    'learning_rate': [0.5,0.6],
    'subsample_for_bin': [20000, 300000, 20000],
    'min_child_samples': [20, 500],
    'reg_alpha': [0.1,0.2,0.3],
    'reg_lambda':[0.05,0.1,0.5],
    'colsample_bytree': [0.6, 1, 10],
    'subsample':[0.8,0.7,0.5],
    'is_unbalance': [True, False],
    'n_jobs' : [-1,-2,-3],
    'random_state' : [50]
}


In [10]:
# Hyperparameter grid
param_grid = {
    'n_estimators': [5000],
    'objective' : ['binary'],
    'class_weight' : ['balanced'],
    'learning_rate': [0.5],
    'reg_alpha': [0.1],
    'reg_lambda':[0.1],
    'random_state' : [50],
      'subsample':[0.8]
}


In [14]:
n_estimators=5000, objective = 'binary',
                            class_weight = 'balanced', learning_rate = 0.05,
                            reg_alpha = 0.1, reg_lambda = 0.1, 
                            subsample = 0.8, n_jobs = -1, random_state = 50)

In [5]:
tb_test_homecredit = pd.read_csv(out_path+"tb_test_homecredit.csv")
tb_test_homecredit.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,STATUS_1_count,STATUS_1_count_norm,STATUS_2_count,STATUS_2_count_norm,STATUS_4_count,STATUS_5_count,STATUS_C_count,STATUS_C_count_norm,STATUS_X_count,STATUS_X_count_norm
0,100001,0,0,1,0,135000.0,568800.0,0.018850,19241,-2329.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.571429
1,100005,0,0,1,0,99000.0,222768.0,0.035792,18064,-4469.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.333333
2,100013,0,1,1,0,202500.0,663264.0,0.019101,20038,-4458.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.500000
3,100028,0,0,1,2,315000.0,1575000.0,0.026392,13976,-1866.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.416667
4,100038,0,1,0,1,180000.0,625500.0,0.010032,13040,-2191.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tb_train_homecredit = pd.read_csv(out_path+"tb_train_homecredit.csv")

In [6]:
tb_train_homecredit.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,STATUS_1_count_norm,STATUS_2_count,STATUS_2_count_norm,STATUS_4_count,STATUS_5_count,STATUS_C_count,STATUS_C_count_norm,STATUS_X_count,STATUS_X_count_norm,TARGET
0,100002,0,0,1,0,202500.0,406597.5,0.018801,9461,-637.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.75,1
1,100003,0,0,0,0,270000.0,1293502.5,0.003541,16765,-1188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0
2,100004,1,1,1,0,67500.0,135000.0,0.010032,19046,-225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0
3,100006,0,0,1,0,135000.0,312682.5,0.008019,19005,-3039.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,100007,0,0,1,0,121500.0,513000.0,0.028663,19932,-3038.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0


In [7]:
X_train = tb_train_homecredit.drop(columns = 'TARGET')

In [8]:
y_train = tb_train_homecredit['TARGET']

In [9]:
import gc
gc.enable()
del tb_train_homecredit
gc.collect()

7

In [13]:
estimator = lgb.LGBMRegressor()

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train, eval_metric = 'auc',
                eval_set = [(valid_features, valid_labels), (X_train, y_train)],
                eval_names = ['valid', 'train'],
                early_stopping_rounds = 100, verbose = 200)

print('Best parameters found by grid search are:', gbm.best_params_)

NameError: name 'valid_features' is not defined

In [18]:
gbm.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [48]:
# Iterate through each fold
for train_indices, valid_indices in folds.split(train_df[feats],train_df['TARGET']):
        
    # Training data for the fold
    train_features, train_labels = train_df[feats].iloc[train_indices], train_df['TARGET'].iloc[train_indices]
    # Validation data for the fold
    valid_features, valid_labels = train_df[feats].iloc[valid_indices], train_df['TARGET'].iloc[valid_indices]
        
    # Create the model
#     model = LGBMClassifier(n_estimators=5000, objective = 'binary', 
#                               class_weight = 'balanced', learning_rate = 0.05, 
#                               reg_alpha = 0.1, reg_lambda = 0.1, 
#                               subsample = 0.8, n_jobs = -1, random_state = 50)
    
    
    estimator = lgb.LGBMRegressor(num_leaves=31)
    
    gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train)

    
        
    # Train the model
    model.fit(train_features, train_labels, eval_metric = 'auc',
                eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                eval_names = ['valid', 'train'],
                early_stopping_rounds = 100, verbose = 200)
    
    oof_preds[valid_indices] = model.predict_proba(valid_features, num_iteration=model.best_iteration_)[:, 1]
    sub_preds += model.predict_proba(test_df[feats], num_iteration=model.best_iteration_)[:, 1] / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = model.feature_importances_
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    del model, train_features, train_labels, valid_features, valid_labels
    gc.collect()

print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))

Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[100]	train's binary_logloss: 0.214511	train's auc: 0.846538	valid's binary_logloss: 0.244271	valid's auc: 0.771202
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[100]	train's binary_logloss: 0.217182	train's auc: 0.848005	valid's binary_logloss: 0.241481	valid's auc: 0.768162
Full AUC score 0.769650


In [ ]:
best_params_

In [44]:
sub_preds

array([0.42190804, 0.66981893, 0.1557249 , ..., 0.12667447, 0.29286942,
       0.68398952])

In [45]:
test_df['TARGET'] = sub_preds
submission = test_df[['SK_ID_CURR', 'TARGET']]
submission.to_csv(out_path+'submissionv2.csv', index= False)

In [46]:
submission

,SK_ID_CURR,TARGET
0,100001,0.421908
1,100005,0.669819
2,100013,0.155725
3,100028,0.220054
4,100038,0.693630
5,100042,0.255233
6,100057,0.075204
7,100065,0.286472
8,100066,0.127594
9,100067,0.533493


In [ ]:
estimator = lgb.LGBMRegressor(num_leaves=31)

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(train_features, train_labels)

print('Best parameters found by grid search are:', gbm.best_params_)